In [1]:
import pandas as pd
import re
import os
import glob

In [2]:
def get_filenames(path):
    '''
    Given a file path with wildcard for extension, capture all filenames.
    '''
    
    file_list = []
    file_list = glob.glob(path)
    
    return file_list   

In [3]:
def get_df_names(path):
    '''
    Given a list of filenames, capture the relevent portion of filename to make dataframe names.
    '''    
    
    df_list = []
    for filename in file_list:
        name = os.path.basename(filename)
        name = name.split('- ')[1]
        name = name.split('_')[0]
        name_df = name.replace(" ", "_") + '_df'
        df_list.append(name_df)
        
    return df_list

In [4]:
def create_df(path, index_number):
    '''
    Given a file path and index number, create a dataframe for the index file with the index dataframe name.
    Create a Source column in the dataframe indicating the Source document. 
    '''   
    file_list = get_filenames(path)
    df_list = get_df_names(path)
    df_name = df_list[index_number]
    file_name = file_list[index_number]
    source_name = file_list[index_number]
    source_name = source_name.split('- ')[1]
    source_name = source_name.split('_')[0]
    
    df_name = pd.read_csv(file_name,low_memory=False,skiprows=[0,2])
    df_name['Source'] = source_name
    
    return df_name

In [5]:
file_list = get_filenames("data/*.csv")

In [6]:
df_list = get_df_names("data/*.csv")

In [7]:
Acquisition_Workforce_df = create_df("data/*.csv", 0)

In [8]:
Control_Group_df = create_df("data/*.csv", 1)

In [9]:
Test_Group_df = create_df("data/*.csv", 2)

In [10]:
def reshape_df(df, index_number, path):
    '''
    Given a dataframe, index number, and path, reshape the specified dataframe from wide to long, with ID Columns
    specified and Value Columns the remaining columns in initial dataframe. 
    '''   

    df_list = get_df_names(path)
    id_cols = {'Source', 'Start Date', 'End Date', 'Response Type', 'IP Address',
                  'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date',
                  'Response ID', 'Recipient Last Name', 'Recipient First Name',
                  'Recipient Email', 'External Data Reference', 'Location Latitude',
                  'Location Longitude', 'Distribution Channel', 'User Language'}
    value_cols = set(df.columns) - id_cols
    melted_df = df_list[index_number]
    
    melted_df = pd.melt(df, 
                       id_vars=list(id_cols), 
                       value_vars=list(value_cols),
                       var_name='Question', 
                       value_name='Response')
    
    return melted_df    

In [11]:
Acquisition_Workforce_melted = reshape_df(Acquisition_Workforce_df,0,"data/*.csv")

In [12]:
Control_Group_melted = reshape_df(Control_Group_df,0,"data/*.csv")

In [13]:
Test_Group_melted = reshape_df(Test_Group_df,0,"data/*.csv")

In [14]:
def merge_df(df1, df2, df3):
    final_df = pd.concat([df1,df2,df3], axis=0, ignore_index=True)
    
    return final_df

In [15]:
final_df = merge_df(Acquisition_Workforce_melted,
        Control_Group_melted,
        Test_Group_melted)

In [16]:
final_df.to_excel("final_output.xlsx", engine='openpyxl')